In [26]:
import numpy as np
import pandas as pd
from operator import attrgetter
from sklearn.linear_model import LogisticRegression
import random

In [27]:
Data = pd.read_excel('./Midterm_Data/Data.xlsx') # training data
Test1 = pd.read_excel('./Midterm_Data/Test1.xlsx') # validation data
Test2 = pd.read_excel('./Midterm_Data/Test2.xlsx') # testing data

Data.fillna(Data.mean(), inplace=True)
# Data # training data

# data preprocessing

- 1 way ratio: 0 ~ 94 (xi)
- 2 way ratio: 95 ~ 4559 (xi * xj)

In [28]:
def data_preprocessing(data):
    
    # Data_bio_ratio
    x = pd.DataFrame(index=range(data.shape[0]),columns=range(95))
    for i in range(19):
        for j in range(5):
            if i < 9:
                x[i * 5 + j] = data[f'F0{i+1}'] / data[f'R0{j+1}']
            else:
                x[i * 5 + j] = data[f'F{i+1}'] / data[f'R0{j+1}']

    one_way = x.to_numpy()
    two_way = np.zeros((data.shape[0], 95 * 94 // 2))
    count = 0
    for i in range(one_way.shape[1]):
        for j in range(i+1, one_way.shape[1]):
            two_way[:, count] = one_way[:, i] * one_way[:, j]
            count += 1
    
    return np.hstack((one_way, two_way)), data[['C01','C02','C03','C04','C05']].to_numpy().T
    return one_way, data[['C01','C02','C03','C04','C05']].to_numpy().T

In [29]:
x_train, y_train = data_preprocessing(Data)
print("x_train.shape: ", x_train.shape)
print("y_train.shape: ", y_train.shape)

x_val, y_val = data_preprocessing(Test1)
print("x_val.shape: ", x_val.shape)
print("y_val.shape: ", y_val.shape)

x_test, y_test = data_preprocessing(Test2)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)


# merge training data and validation data
x_train_val = np.vstack((x_train, x_val))
y_train_val = np.hstack((y_train, y_val))
print("x_train_val.shape: ", x_train_val.shape)
print("y_train_val.shape: ", y_train_val.shape)

x_train.shape:  (4240, 4560)
y_train.shape:  (5, 4240)
x_val.shape:  (475, 4560)
y_val.shape:  (5, 475)
x_test.shape:  (987, 4560)
y_test.shape:  (5, 987)
x_train_val.shape:  (4715, 4560)
y_train_val.shape:  (5, 4715)


# Genetic Programming

In [46]:
class individual:
    def __init__(self, bio_lst, op_lst):
        self.biomarker_chromosome = bio_lst
        self.operator_chromosome = op_lst
        self.fitness = 0

In [47]:
def calculate(num1, num2, oper):
#     oper = 0 -> +
#     oper = 1 -> -
#     oper = 2 -> *
#     oper = 3 -> /

    if oper == 0:
        return (num1+num2)
    
    elif oper == 1:
        return (num1-num2)
    
    elif oper == 1:
        return (num1*num2)
    
    else:
        return (num1/num2)

In [48]:
def formula(bio_1, bio_2, bio_3, bio_4, op1, op2, op3, x):
    
    tmp_1 = calculate(x[bio_1], x[bio_2], op1)
    tmp_2 = calculate(x[bio_3], x[bio_4], op2)
    
    return calculate(tmp_1, tmp_2, op3)

In [49]:
def identify_class(x, ind):
    # if 1 == true:
    if formula(ind.biomarker_chromosome[0], ind.biomarker_chromosome[1],\
               ind.biomarker_chromosome[2], ind.biomarker_chromosome[3],\
               ind.operator_chromosome[0], ind.operator_chromosome[1], ind.operator_chromosome[2], x) < \
       formula(ind.biomarker_chromosome[4], ind.biomarker_chromosome[5],\
               ind.biomarker_chromosome[6], ind.biomarker_chromosome[7],\
               ind.operator_chromosome[3], ind.operator_chromosome[4], ind.operator_chromosome[5], x):

        # if 2 == true:
        if formula(ind.biomarker_chromosome[8], ind.biomarker_chromosome[9],\
                   ind.biomarker_chromosome[10], ind.biomarker_chromosome[11],\
                   ind.operator_chromosome[6], ind.operator_chromosome[7], ind.operator_chromosome[8], x) < \
           formula(ind.biomarker_chromosome[12], ind.biomarker_chromosome[13],\
                   ind.biomarker_chromosome[14], ind.biomarker_chromosome[15],\
                   ind.operator_chromosome[9], ind.operator_chromosome[10], ind.operator_chromosome[11], x):
            # cla = 4
            return 4

        # else: 
        else:
            # cla = 5
            return 5

    # else (1 == false):
    else:


        # if 3 == true:
        if formula(ind.biomarker_chromosome[16], ind.biomarker_chromosome[17],\
                   ind.biomarker_chromosome[18], ind.biomarker_chromosome[19],\
                   ind.operator_chromosome[12], ind.operator_chromosome[13], ind.operator_chromosome[14], x) < \
           formula(ind.biomarker_chromosome[20], ind.biomarker_chromosome[21],\
                   ind.biomarker_chromosome[22], ind.biomarker_chromosome[23],\
                   ind.operator_chromosome[15], ind.operator_chromosome[16], ind.operator_chromosome[17], x):

            # if 4 == true:
            if formula(ind.biomarker_chromosome[24], ind.biomarker_chromosome[25],\
                       ind.biomarker_chromosome[26], ind.biomarker_chromosome[27],\
                       ind.operator_chromosome[18], ind.operator_chromosome[19], ind.operator_chromosome[20], x) < \
               formula(ind.biomarker_chromosome[28], ind.biomarker_chromosome[29],\
                       ind.biomarker_chromosome[30], ind.biomarker_chromosome[31],\
                       ind.operator_chromosome[21], ind.operator_chromosome[22], ind.operator_chromosome[23], x):
                # cla = 1
                return 1


            # else (4 == false): 
            else:
                # cla = 2
                return 2


        # else (3 == false):
        else:

            # if 5 == true:
            if formula(ind.biomarker_chromosome[32], ind.biomarker_chromosome[33],\
                       ind.biomarker_chromosome[34], ind.biomarker_chromosome[35],\
                       ind.operator_chromosome[24], ind.operator_chromosome[25], ind.operator_chromosome[26], x) < \
               formula(ind.biomarker_chromosome[36], ind.biomarker_chromosome[37],\
                       ind.biomarker_chromosome[38], ind.biomarker_chromosome[39],\
                       ind.operator_chromosome[27], ind.operator_chromosome[28], ind.operator_chromosome[29], x):
                # cla = 3
                return 3


            # else (5 == false): 
            else:
                # cla = all 0
                return 0

In [50]:
def y_to_ans(y):
    if (y == [0, 0, 0, 0, 0]).all():
        return 0
    
    elif (y == [1, 0, 0, 0, 0]).all():
        return 1

    elif (y == [0, 1, 0, 0, 0]).all():
        return 2
    
    elif (y == [0, 0, 1, 0, 0]).all():
        return 3
    
    elif (y == [0, 0, 0, 1, 0]).all():
        return 4
    
    elif (y == [0, 0, 0, 0, 1]).all():
        return 5

In [51]:
def calculate_fitness(ind):
    fitness = 0
    
    for i in range(len(x_train_val)):
        x_class = identify_class(x_train_val[i], ind)
        ans = y_to_ans(y_train_val[:, i])
        
        if ans == x_class:
            fitness += 1
            
    return fitness

In [52]:
# len(parentssss) == len(offspringsss)
def breed(parentssss):
    
    offspringsss = []
    bio_chromosome_length = len(parentssss[0].biomarker_chromosome)
    op_chromosome_length = len(parentssss[0].operator_chromosome)
    
    for i in range(int(len(parentssss) / 2)):
        parent1 = random.choice(parentssss)
        parent2 = random.choice(parentssss)
        
        # crossover
        
        # bio part
        # which_parent = np.random.uniform(size = bio_chromosome_length)
        cut_point = np.random.choice(bio_chromosome_length)
        
        bio_offspring1 = np.zeros(bio_chromosome_length, dtype=int)
        bio_offspring2 = np.zeros(bio_chromosome_length, dtype=int)
        
        for k in range(bio_chromosome_length):
            if k > cut_point:
                bio_offspring1[k] = parent1.biomarker_chromosome[k]
                bio_offspring2[k] = parent2.biomarker_chromosome[k]
            else:
                bio_offspring1[k] = parent2.biomarker_chromosome[k]
                bio_offspring2[k] = parent1.biomarker_chromosome[k]
            
            
        # mutation
        prob = np.random.uniform()
        mutation_gene_index = np.random.randint(bio_chromosome_length)
        mutation_gene_value = np.random.choice(x_train.shape[1])
        if prob < 0.5:
            bio_offspring1[mutation_gene_index] = mutation_gene_value
        elif prob > 0.5:
            bio_offspring2[mutation_gene_index] = mutation_gene_value
        
                
        # op part
#         which_parent = np.random.uniform(size = op_chromosome_length)
        cut_point = np.random.choice(op_chromosome_length)
        

        op_offspring1 = np.zeros(op_chromosome_length, dtype=int)
        op_offspring2 = np.zeros(op_chromosome_length, dtype=int)
        
        for k in range(op_chromosome_length):
            if k > cut_point:
                op_offspring1[k] = parent1.operator_chromosome[k]
                op_offspring2[k] = parent2.operator_chromosome[k]
            else:
                op_offspring1[k] = parent2.operator_chromosome[k]
                op_offspring2[k] = parent1.operator_chromosome[k]
        
        # mutation
        prob = np.random.uniform()
        mutation_gene_index = np.random.randint(op_chromosome_length)
        mutation_gene_value = np.random.choice(4)
        if prob < 0.5:
            op_offspring1[mutation_gene_index] = mutation_gene_value
        elif prob > 0.5:
            op_offspring2[mutation_gene_index] = mutation_gene_value

                
                
        offspringsss.append(individual(list(bio_offspring1), list(op_offspring1)))
        offspringsss.append(individual(list(bio_offspring2), list(op_offspring2)))
    
    
    return offspringsss

In [53]:
def pop_avg_fitness(pop):
    
    total = 0
    for i in range(len(pop)):
        total += pop[i].fitness
        
    return total/len(pop)

In [55]:
biomarker_chromosome_ell = 40
operator_chromosome_ell = 30
pop_size = 10
generation = 100

population = []
pop_avg_fitness_lst = []

best = individual([], [])
best.fitness = 0


# initilize first generation population
for i in range(pop_size):
    tmp_bio_chromosome = np.random.choice(list(range(x_train.shape[1])), biomarker_chromosome_ell, replace=True)

    # 0 -> +, 1 -> -, 2 -> *, 3 -> /
    tmp_op_chromosome = np.random.choice(list(range(4)), operator_chromosome_ell, replace=True)
    
    tmp = individual(tmp_bio_chromosome, tmp_op_chromosome)
    tmp.fitness = calculate_fitness(tmp)

    population.append(tmp)

    
for iteration in range(generation):
    
    offsprings = breed(population)
    
    for i in range(pop_size):
        offsprings[i].fitness = calculate_fitness(offsprings[i])
    

    # selection
    population += offsprings
    population.sort(key=lambda x: x.fitness, reverse=True)
    population = population[:pop_size]
    
    
    tmp = pop_avg_fitness(population)
    print(f'{iteration}th avg: {tmp}')
    pop_avg_fitness_lst.append(tmp)
    
    
    # record best solution
    if population[0].fitness > best.fitness:
        best = population[0]
        print(f'best.fitness: {best.fitness}')
    

0th avg: 900.1
best.fitness: 1009
1th avg: 970.6
best.fitness: 1211
2th avg: 1002.0
3th avg: 1071.1
best.fitness: 1218
4th avg: 1166.6
best.fitness: 1604
5th avg: 1246.4
6th avg: 1364.1
best.fitness: 1629
7th avg: 1415.7
8th avg: 1547.9
9th avg: 1609.4
10th avg: 1612.6
11th avg: 1616.3
12th avg: 1624.7
best.fitness: 1644
13th avg: 1630.7
14th avg: 1633.5
15th avg: 1634.0
16th avg: 1636.6
17th avg: 1639.0
18th avg: 1639.2
19th avg: 1640.4
20th avg: 1641.5
21th avg: 1656.3
best.fitness: 1768
22th avg: 1669.3
23th avg: 1671.6
24th avg: 1672.4
25th avg: 1686.4
26th avg: 1703.1
27th avg: 1733.8
28th avg: 1762.4
29th avg: 1774.7
best.fitness: 1838
30th avg: 1782.5
best.fitness: 1843
31th avg: 1782.5
32th avg: 1796.6
33th avg: 1823.9
34th avg: 1838.7
35th avg: 1840.4
36th avg: 1844.5
best.fitness: 1858
37th avg: 1844.5
38th avg: 1846.5
best.fitness: 1863
39th avg: 1848.5
40th avg: 1850.5
41th avg: 1855.9
42th avg: 1863.0
43th avg: 1863.0
44th avg: 1863.0
45th avg: 1863.0
46th avg: 1863.0
47th

KeyboardInterrupt: 

# print best chromosome, best acc, and predict test2.xls

In [31]:
with open('./gp_avg_fitness_lst.txt', 'w') as f:
    for i in pop_avg_fitness_lst:
        f.write("%s\n" % i)
        
    f.write("best bio: %s\n" % best.biomarker_chromosome)
    f.write("best op: %s\n" % best.operator_chromosome)
    f.write("best fitness: %s\n" % best.fitness)
    

In [77]:
def two_ratio(num):
    if num < 95:
        return 0, num
    
    i = 1
    k = 94
    num = num - 95
    while num > k:
        
        num = num - k
        i += 1
        k -= 1
        
    return i, num + i

In [80]:
best_op_chromosome = open('./result/gp_avg_fitness_lst.txt', 'r').readlines()[-2][10:-2].split(',')
best_op_chromosome = list(map(int, best_op_chromosome))
print(best_op_chromosome)

best_bio_chromosome = open('./result/gp_avg_fitness_lst.txt', 'r').readlines()[-3][11:-2].split(',')
best_bio_chromosome = list(map(int, best_bio_chromosome))
print(best_bio_chromosome)


for i in best_bio_chromosome:
    print(two_ratio(i))

[0, 1, 0, 1, 0, 0, 0, 1, 0, 2, 3, 0, 3, 0, 1, 0, 1, 3, 1, 1, 1, 1, 3, 3, 0, 0, 1, 3, 2, 1]
[3156, 4329, 3963, 378, 2246, 1520, 1801, 2088, 1530, 3141, 2198, 2698, 2385, 804, 165, 2395, 789, 3842, 3900, 1176, 896, 3425, 178, 3406, 2236, 3944, 616, 3062, 81, 3327, 946, 1272, 2458, 2025, 374, 1678, 198, 3973, 1083, 1453]
(42, 69)
(73, 95)
(60, 93)
(4, 8)
(27, 59)
(17, 58)
(21, 37)
(25, 38)
(17, 68)
(42, 54)
(26, 79)
(34, 63)
(29, 65)
(8, 80)
(1, 71)
(29, 75)
(8, 65)
(57, 80)
(59, 65)
(13, 32)
(9, 86)
(47, 88)
(1, 84)
(47, 69)
(27, 49)
(60, 74)
(6, 67)
(40, 82)
(0, 81)
(45, 87)
(10, 51)
(14, 47)
(30, 73)
(24, 45)
(3, 95)
(19, 63)
(2, 11)
(61, 69)
(12, 21)
(16, 69)


In [111]:
best.biomarker_chromosome = best_bio_chromosome
best.operator_chromosome = best_op_chromosome

for i in range(len(x_test)):
    if identify_class(x_test[i], best) == 0:
        Test2.loc[i][-5:] = [0,0,0,0,0]
    elif identify_class(x_test[i], best) == 1:
        Test2.loc[i][-5:] = [1,0,0,0,0]
    elif identify_class(x_test[i], best) == 2:
        Test2.loc[i][-5:] = [0,1,0,0,0]
    elif identify_class(x_test[i], best) == 3:
        Test2.loc[i][-5:] = [0,0,1,0,0]
    elif identify_class(x_test[i], best) == 4:
        Test2.loc[i][-5:] = [0,0,0,1,0]
    elif identify_class(x_test[i], best) == 5:
        Test2.loc[i][-5:] = [0,0,0,0,1]
        
Test2.to_excel("./Midterm_Data/Test2_gp_result.xlsx", index=False) 